In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pandas as pd
import keras_tuner as kt
import tensorflow_datasets as tfds
import time

import sys
import os
py_file_location = "/content/drive/My Drive"
sys.path.append(os.path.abspath('/content/drive/MyDrive/NeuralNetwork'))
from models_hs import *
from trainer import *
from preprocess import *
from callbacks import *


# 0 Data

Read in data for each dimension and store as `pd.DataFrame`:

In [10]:
def get_dfs(dims, num_samples):
  dataframes_list = []
  for dim in dims:
      temp_df = pd.read_csv(f"/content/drive/MyDrive/data/{num_samples}_basket_data_{dim}.csv")
      temp_df = temp_df.drop(['Unnamed: 0'], axis=1)

      # move column with contract prices to the end
      cols = list(temp_df.columns.values) 
      cols.pop(cols.index('contract_price')) 
      temp_df = temp_df[cols+['contract_price']] 

      dataframes_list.append(temp_df)
  return dataframes_list

In [11]:
dims = [1, 4, 7, 10, 13, 16]
dataframes_list = get_dfs(dims, 1000)
dataframes_list[0]

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,Underlying_0,Rho_0,contract_price
0,52.0,16.50,0.010564,0.041048,1.775966,0.058849,0.015002,17.332500,-0.333905,10.938264
1,21.0,49.50,0.039117,0.049353,1.540456,0.489135,0.001440,9.260000,-0.129997,0.571392
2,31.0,14.00,0.029249,0.006491,0.982883,0.224651,0.017221,16.414301,-0.462886,8.445650
3,18.0,8.70,0.038737,0.016305,2.372708,0.329505,0.006641,15.892000,-0.141496,9.129831
4,59.0,16.50,0.043640,0.038880,4.562963,0.706960,0.053901,11.277000,-0.379364,10.687387
...,...,...,...,...,...,...,...,...,...,...
995,38.0,13.00,0.001502,0.046531,1.663993,0.145838,0.001362,12.819120,-0.307483,6.298129
996,85.0,15.00,0.042181,0.044124,0.302489,0.661890,0.018707,18.106250,-0.050286,24.277845
997,29.0,0.48,0.033398,0.005511,4.443828,0.506015,0.002109,67.483701,-0.693189,65.758164
998,7.0,9.50,0.004519,0.036562,2.277263,0.288585,0.003034,15.919620,-0.470891,7.345720


Test model on 10000 samples with dim = 1:

In [12]:
dataframes_list_1 = get_dfs([1], 10000)
dataframes_list_1[0]

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,Underlying_0,Rho_0,contract_price
0,24.0,4.250,0.002046,0.050490,0.842923,0.631400,0.022602,58.010400,-0.320407,61.992674
1,119.0,5.050,0.050203,0.048482,2.676144,0.702256,0.001616,39.998899,-0.102169,16.377944
2,10.0,15.000,0.012146,0.009763,0.952558,0.693369,0.009901,5.080200,-0.394712,0.026545
3,235.0,14.000,0.028765,0.038385,4.946453,0.012569,0.001688,13.283750,-0.471352,0.000000
4,130.0,16.300,0.010112,0.045151,0.474549,0.218541,0.015604,4.440500,-0.101970,4.166032
...,...,...,...,...,...,...,...,...,...,...
9995,22.0,3.975,0.003293,0.047075,0.377678,0.601305,0.026390,12.496370,-0.421565,11.578677
9996,7.0,14.000,0.022504,0.025393,0.826864,0.429884,0.001874,15.524250,-0.229666,3.513765
9997,10.0,12.400,0.002174,0.040227,0.190017,0.502809,0.002174,4.612700,-0.243227,0.296504
9998,192.0,4.450,0.044043,0.030118,4.034105,0.391629,0.025649,19.868500,-0.190983,2.275450


# 1 Training

In [89]:
import argparse
from pyexpat import model
import numpy as np
import tensorflow as tf

def model_builder_hs(dim,
            num_layers   = 2,
            hidden_units = [14,7],
            output_shape = (1,),
            activation = 'elu',
            regularizer = None,
            initializer = tf.keras.initializers.he_uniform(),
            final_activation = 'linear',
            dropout = None,
            batchnorm = False
            ): 
    """
    Returns a model for training and testing.  

    Args:
        - dim: int, basket size
        - num_layers: int, number of hidden layers
        - hidden_units: list of number of hidden units in each layer
        - output_shape: shape of the output data
        - activation: string, activation function
        - initializer: initializer for the weights
        - final_activation: string, activation function of final layer
        - dropout: list, dropout rate for each layer, default None
        - batchnorm: bool, specifies if batch normalization is used, default False 
    
    Output:  
        - model: tf.keras.Model, compiled if compile is True
    """  
    assert num_layers == len(hidden_units), "Number of hidden units must match number of layers"
    if dropout is not None:  
        assert num_layers == len(dropout), "Number of dropout rates must match number of layers"

    input_shape = (7 + 2*dim,)

    inputs = tf.keras.layers.Input(shape=input_shape)
    h = tf.keras.layers.Flatten()(inputs)

    for i, layer in enumerate(hidden_units):
        h = tf.keras.layers.Dense(layer, activation=activation, kernel_regularizer= regularizer,
                                  kernel_initializer = initializer)(h)
        if dropout:
            h = tf.keras.layers.Dropout(dropout[i])(h)
        if batchnorm:
            h = tf.keras.layers.BatchNormalization()(h)
    if final_activation is not None:
        outputs = tf.keras.layers.Dense(output_shape[0], activation=final_activation,
                                        kernel_initializer = initializer)(h)
    else:
        outputs = tf.keras.layers.Dense(output_shape[0], 
                                        kernel_initializer = initializer)(h)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)  

    return model   


def tuned_model_hs(hp):
    """
    Returns a compiled hyperModel for keras tuner. 

    """  

    # defining a set of hyperparameters for tuning and a range of values for each
    num_layers = hp.Int('num_layers', min_value=1, max_value=5) 
    activation = hp.Choice('activation', ['elu','tanh', 'relu', 'sigmoid'])
    learning_rate = hp.Float('learning_rate', min_value=10**(-3), max_value=0.01)
    rate_decay = hp.Float('rate_decay', min_value=0.85, max_value=0.9995)
    l1_reg = hp.Float('l1_regularizer', min_value=10**(-8), max_value=10**(-6.5))
    l2_reg = hp.Float('l1_regularizer', min_value=10**(-8), max_value=10**(-6.5))
    batchnorm = hp.Boolean(name = 'batchnorm')
    
    hidden_units, dropouts = [],[]
    for i in range(num_layers):
        hidden_unit = hp.Int(f'units_{i+1}', min_value=5, max_value=7)
        hidden_units.append(hidden_unit)
        dropout = hp.Float(f'dropout_{i+1}', min_value=0.0, max_value=0.5, step=0.1)
        dropouts.append(dropout)

    model = model_builder_hs(dim,
                    num_layers = num_layers, 
                    hidden_units = hidden_units,
                    dropout = dropouts,
                    activation = activation,
                    batchnorm = batchnorm,
                    regularizer = tf.keras.regularizers.l1_l2(l1_reg,l2_reg)
                    )

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate, decay_steps = 4000, decay_rate = rate_decay, staircase = True)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule), loss = tf.keras.losses.MeanAbsolutePercentageError(), 
                  metrics = [tf.keras.metrics.MeanSquaredError()])

    return model  

## 1.1 Using Random Tuner:

In [90]:
dims = [1, 4, 7, 10, 13, 16]
best_models_history_rn = []
best_hp_lists_rn = []
begin_train_rn, end_train_rn = [0]*len(dims),[0]*len(dims)

for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list[i].to_numpy(), scaling=True, prefetch=True)

    print(f"Start training for basket option with size {dim}:")

    random_tuner = kt.RandomSearch(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_trials=3, # the maximum number of trials to run
    executions_per_trial=2, # the number of models generated on each trial
    overwrite=True, # whether to overwrite previous trials
    directory="hyperparams/RandomSearch", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    random_tuner.search(train_ds, epochs = 5, validation_data = valid_ds)    

    best_hp_lists_rn.append(random_tuner.get_best_hyperparameters(1)[0])
    models = random_tuner.get_best_models(num_models=1)
    best_model = models[0]
    
    begin_train_rn[i] = time.time()
    history = best_model.fit(train_ds, epochs = 10, validation_data = valid_ds)
    end_train_rn[i] = time.time()
    best_models_history_rn.append(history)


Trial 3 Complete [00h 00m 04s]
val_loss: 460753.625

Best val_loss So Far: 460753.625
Total elapsed time: 00h 00m 12s
Epoch 1/10
25/25 [==============================] - 1s 9ms/step - loss: 10146371.0000 - mean_squared_error: 0.1016 - val_loss: 4666153.0000 - val_mean_squared_error: 0.0273
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 7636520.5000 - mean_squared_error: 0.0827 - val_loss: 156027.7656 - val_mean_squared_error: 0.0046
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 6155944.5000 - mean_squared_error: 0.0446 - val_loss: 1498681.0000 - val_mean_squared_error: 0.0023
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 6680031.5000 - mean_squared_error: 0.0378 - val_loss: 1119015.5000 - val_mean_squared_error: 0.0026
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 4636922.0000 - mean_squared_error: 0.0265 - val_loss: 2540093.5000 - val_mean_squared_error: 0.0026
Epoch 6/10
25/25 [====

The best hyperparameters:

In [91]:
pd.DataFrame([best_hp_lists_rn[i].values for i in range(len(dims))], index = dims)

,num_layers,activation,learning_rate,rate_decay,l1_regularizer,batchnorm,units_1,dropout_1,units_2,dropout_2,units_3,dropout_3,units_4,dropout_4,units_5,dropout_5
1,5,sigmoid,0.003218,0.869513,9.561609e-08,False,7,0.2,5,0.1,7,0.4,5.0,0.4,5.0,0.0
4,2,elu,0.005152,0.973258,2.327540e-07,False,5,0.3,7,0.1,5,0.3,7.0,0.0,NaN,NaN
7,3,tanh,0.008316,0.897428,2.257280e-07,True,7,0.0,5,0.0,5,0.0,NaN,NaN,NaN,NaN
10,4,tanh,0.004704,0.929727,2.355205e-07,True,5,0.4,6,0.1,7,0.1,5.0,0.0,NaN,NaN
13,2,relu,0.001846,0.868290,8.067674e-08,False,6,0.1,6,0.4,6,0.3,NaN,NaN,NaN,NaN
16,3,sigmoid,0.005298,0.920690,2.470968e-07,False,5,0.3,6,0.5,6,0.4,5.0,0.2,6.0,0.4


In [92]:
for i,dim in enumerate(dims):
  print(f"The best validation MSE for basket of size {dim}: \n  {min(best_models_history_rn[i].history['val_mean_squared_error'])}")
  # print(f"The best validation MAPE for basket of size {dim}: \n  {min(best_models_history[i].history['val_mean_absolute_percentage_error'])}")

The best validation MSE for basket of size 1: 
  0.0015863908920437098
The best validation MSE for basket of size 4: 
  0.008871371857821941
The best validation MSE for basket of size 7: 
  0.023466937243938446
The best validation MSE for basket of size 10: 
  0.0013349620858207345
The best validation MSE for basket of size 13: 
  0.013999377377331257
The best validation MSE for basket of size 16: 
  0.0023309816606342793


In [88]:
for i,dim in enumerate(dims):
  print(f"Training time for dim {dim}: {end_train_rn[i] - begin_train_rn[i]} seconds")

Training time for dim 1: 3.123819589614868 seconds
Training time for dim 4: 4.965630531311035 seconds
Training time for dim 7: 4.7338645458221436 seconds
Training time for dim 10: 2.6725995540618896 seconds
Training time for dim 13: 2.86281156539917 seconds
Training time for dim 16: 2.6475534439086914 seconds


## 1.2 Using Hyperband Tuner:


In [95]:
dims = [1, 4, 7, 10, 13, 16]
best_models_history_hb = []
best_hp_lists_hb = []
begin_train_hb, end_train_hb = [0]*len(dims),[0]*len(dims)

for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list[i].to_numpy(), scaling=True, prefetch=True)

    print(f"Start training for basket option with size {dim}:")

    Hyperband_tuner = kt.Hyperband(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_epochs=10,
    factor=3,
    directory="hyperparams/HyperbandSearch", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    Hyperband_tuner.search(train_ds, epochs = 5, validation_data = valid_ds)    

    best_hp_lists_hb.append(Hyperband_tuner.get_best_hyperparameters(1)[0])
    models = Hyperband_tuner.get_best_models(num_models=1)
    best_model = models[0]
    
    begin_train_hb[i] = time.time()
    history = best_model.fit(train_ds, epochs = 10, validation_data = valid_ds)
    end_train_hb[i] = time.time()
    best_models_history_hb.append(history)


Trial 30 Complete [00h 00m 03s]
val_loss: 447959.59375

Best val_loss So Far: 38129.68359375
Total elapsed time: 00h 01m 17s
Epoch 1/10
25/25 [==============================] - 1s 9ms/step - loss: 271396.0000 - mean_squared_error: 0.0061 - val_loss: 51241.2266 - val_mean_squared_error: 0.0034
Epoch 2/10
25/25 [==============================] - 0s 3ms/step - loss: 139501.4844 - mean_squared_error: 0.0060 - val_loss: 286670.0625 - val_mean_squared_error: 0.0031
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 195842.6406 - mean_squared_error: 0.0059 - val_loss: 370529.0000 - val_mean_squared_error: 0.0030
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 144811.7500 - mean_squared_error: 0.0058 - val_loss: 265202.6875 - val_mean_squared_error: 0.0039
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 216428.5000 - mean_squared_error: 0.0059 - val_loss: 424619.1562 - val_mean_squared_error: 0.0041
Epoch 6/10
25/25 [========

Best hyperparameters:

In [99]:
pd.DataFrame([best_hp_lists_hb[i].values for i in range(len(dims))], index = dims)

,num_layers,activation,learning_rate,rate_decay,l1_regularizer,batchnorm,units_1,dropout_1,units_2,dropout_2,...,dropout_3,units_4,dropout_4,units_5,dropout_5,tuner/epochs,tuner/initial_epoch,tuner/bracket,tuner/round,tuner/trial_id
1,2,relu,0.005872,0.961157,2.460751e-07,False,7,0.2,6,0.3,...,0.3,6,0.3,5.0,0.4,10,4,1,1,0019
4,5,relu,0.007870,0.918098,2.744735e-07,False,7,0.0,6,0.3,...,0.1,5,0.0,7.0,0.1,4,2,2,1,0009
7,3,relu,0.004759,0.859409,2.223006e-07,False,5,0.3,6,0.1,...,0.2,6,0.1,6.0,0.4,10,4,1,1,0023
10,5,sigmoid,0.001955,0.860591,1.043340e-07,False,6,0.0,6,0.4,...,0.3,7,0.0,6.0,0.2,10,0,0,0,NaN
13,4,relu,0.005957,0.967706,1.358656e-07,False,5,0.0,5,0.2,...,0.2,7,0.4,NaN,NaN,10,0,0,0,NaN
16,2,relu,0.005348,0.908735,1.606016e-07,False,5,0.2,6,0.1,...,0.0,7,0.3,6.0,0.0,4,0,1,0,NaN


In [100]:
for i,dim in enumerate(dims):
  print(f"The best validation MSE for basket of size {dim}: \n  {min(best_models_history_hb[i].history['val_mean_squared_error'])}")
  # print(f"The best validation MAPE for basket of size {dim}: \n  {min(best_models_history[i].history['val_mean_absolute_percentage_error'])}")

The best validation MSE for basket of size 1: 
  0.0004873159050475806
The best validation MSE for basket of size 4: 
  0.005056868307292461
The best validation MSE for basket of size 7: 
  0.03680519759654999
The best validation MSE for basket of size 10: 
  6.294813647400588e-05
The best validation MSE for basket of size 13: 
  0.018639778718352318
The best validation MSE for basket of size 16: 
  0.002898908220231533


In [101]:
for i,dim in enumerate(dims):
  print(f"Training time for dim {dim}: {end_train_hb[i] - begin_train_hb[i]} seconds")

Training time for dim 1: 2.1893463134765625 seconds
Training time for dim 4: 2.9390745162963867 seconds
Training time for dim 7: 2.420423746109009 seconds
Training time for dim 10: 4.247692584991455 seconds
Training time for dim 13: 2.458421230316162 seconds
Training time for dim 16: 2.228080987930298 seconds


## 1.3 Using Bayesian Optimization

In [110]:
dims = [1, 4, 7, 10, 13, 16]
best_models_history_bo = []
best_hp_lists_bo = []
begin_train_bo, end_train_bo = [0]*len(dims),[0]*len(dims)

for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list[i].to_numpy(), scaling=True, prefetch=True)

    print(f"Start training for basket option with size {dim}:")

    bayes_tuner = kt.BayesianOptimization(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_trials=3,
    directory="hyperparams/BayesianOptimization", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    bayes_tuner.search(train_ds, epochs = 5, validation_data = valid_ds)    

    best_hp_lists_bo.append(bayes_tuner.get_best_hyperparameters(1)[0])
    models = bayes_tuner.get_best_models(num_models=1)
    best_model = models[0]
    
    begin_train_bo[i] = time.time()
    history = best_model.fit(train_ds, epochs = 10, validation_data = valid_ds)
    end_train_bo[i] = time.time()
    best_models_history_bo.append(history)


(800, 10) (100, 10) (100, 10)
Start training for basket option with size 1:
Epoch 1/10
25/25 [==============================] - 2s 13ms/step - loss: 1751156.0000 - mean_squared_error: 0.0101 - val_loss: 2384749.5000 - val_mean_squared_error: 0.0062
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 2626196.5000 - mean_squared_error: 0.0088 - val_loss: 1348370.8750 - val_mean_squared_error: 0.0035
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 1976045.2500 - mean_squared_error: 0.0095 - val_loss: 1906662.2500 - val_mean_squared_error: 0.0044
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 1620164.5000 - mean_squared_error: 0.0085 - val_loss: 929981.3750 - val_mean_squared_error: 0.0036
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 1952873.6250 - mean_squared_error: 0.0070 - val_loss: 1280106.3750 - val_mean_squared_error: 0.0036
Epoch 6/10
25/25 [==============================] - 0s 4ms/step 

Best hyperparameters:

In [114]:
pd.DataFrame([best_hp_lists_bo[i].values for i in range(len(dims))], index = dims)

,num_layers,activation,learning_rate,rate_decay,l1_regularizer,batchnorm,units_1,dropout_1,units_2,dropout_2,units_3,dropout_3,units_4,dropout_4
1,4,relu,0.006859,0.941751,2.889118e-07,True,6,0.3,5,0.0,5.0,0.0,5.0,0.0
4,2,sigmoid,0.009804,0.903555,2.336900e-07,False,6,0.4,5,0.0,NaN,NaN,NaN,NaN
7,2,relu,0.006772,0.889963,2.878761e-07,False,6,0.3,5,0.0,NaN,NaN,NaN,NaN
10,1,relu,0.005080,0.949280,8.696250e-08,True,7,0.5,5,0.2,7.0,0.4,7.0,0.1
13,2,relu,0.007673,0.884123,2.020740e-07,False,6,0.5,5,0.3,6.0,0.4,5.0,0.5
16,4,elu,0.009133,0.975671,1.267550e-07,False,6,0.4,5,0.0,5.0,0.0,5.0,0.0


In [115]:
for i,dim in enumerate(dims):
  print(f"The best validation MSE for basket of size {dim}: \n  {min(best_models_history_bo[i].history['val_mean_squared_error'])}")
  # print(f"The best validation MAPE for basket of size {dim}: \n  {min(best_models_history_bo[i].history['val_mean_absolute_percentage_error'])}")

The best validation MSE for basket of size 1: 
  0.003528574714437127
The best validation MSE for basket of size 4: 
  0.017625294625759125
The best validation MSE for basket of size 7: 
  0.05020308122038841
The best validation MSE for basket of size 10: 
  9.148690878646448e-05
The best validation MSE for basket of size 13: 
  0.01464453898370266
The best validation MSE for basket of size 16: 
  0.004053478594869375


In [116]:
for i,dim in enumerate(dims):
  print(f"Training time for dim {dim}: {end_train_bo[i] - begin_train_bo[i]} seconds")

Training time for dim 1: 3.450819969177246 seconds
Training time for dim 4: 2.2184031009674072 seconds
Training time for dim 7: 2.136911392211914 seconds
Training time for dim 10: 2.1807596683502197 seconds
Training time for dim 13: 2.1221179962158203 seconds
Training time for dim 16: 2.3997867107391357 seconds
